<a href="https://colab.research.google.com/github/shrapple/deep-learning-challenge/blob/main/Starter_Code_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [14]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [15]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis = 1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [16]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [17]:
# Look at APPLICATION_TYPE value counts for binning
application_types =application_df['APPLICATION_TYPE'].value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [18]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
applications = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = applications[applications < cutoff_value].index
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [19]:
ask_amnt =application_df['ASK_AMT'].value_counts()
ask_amnt

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [20]:
# Calculate the Z-score for the 'ASK_AMT' column
z_scores = (application_df['ASK_AMT'] - application_df['ASK_AMT'].mean()) / application_df['ASK_AMT'].std()

# Define a threshold to identify outliers (for example, z-score greater than 3)
outlier_threshold = 3

# Filter out rows with Z-scores above the threshold
application_df = application_df[(z_scores < outlier_threshold) & (z_scores > -outlier_threshold)]

# Check the distribution of ASK_AMT after removing outliers
application_df['ASK_AMT'].describe()


count    3.424600e+04
mean     6.828438e+05
std      7.955398e+06
min      5.000000e+03
25%      5.000000e+03
50%      5.000000e+03
75%      7.246250e+03
max      2.611646e+08
Name: ASK_AMT, dtype: float64

In [21]:
# Calculate the Q1 (25th percentile) and Q3 (75th percentile) values of the 'ASK_AMT' column
Q1 = application_df['ASK_AMT'].quantile(0.25)
Q3 = application_df['ASK_AMT'].quantile(0.75)

# Calculate the IQR (Interquartile Range)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
application_df = application_df[(application_df['ASK_AMT'] >= lower_bound) & (application_df['ASK_AMT'] <= upper_bound)]

# Check the distribution of ASK_AMT after removing outliers
application_df['ASK_AMT'].describe()

count    25999.000000
mean      5059.789300
std        461.978384
min       5000.000000
25%       5000.000000
50%       5000.000000
75%       5000.000000
max      10607.000000
Name: ASK_AMT, dtype: float64

In [22]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    12505
C2000     4681
C1200     3985
C2100     1617
C3000     1532
         ...  
C2380        1
C1732        1
C1728        1
C4120        1
C2150        1
Name: CLASSIFICATION, Length: 64, dtype: int64

In [23]:
# You may find it helpful to look at CLASSIFICATION value counts >1
limiter = 1
applications_class = application_df['CLASSIFICATION'].value_counts()
classy = applications_class[applications_class > limiter]
print(classy)


C1000    12505
C2000     4681
C1200     3985
C2100     1617
C3000     1532
C7000      542
C1700      222
C4000      129
C5000      106
C1270       89
C2700       75
C7100       62
C2800       61
C1280       46
C1300       42
C1230       35
C2300       28
C1240       28
C1400       26
C7200       20
C6000       14
C1250       13
C8000       13
C7120       11
C1278       10
C8200        9
C1238        9
C1237        9
C1235        9
C1500        7
C1720        6
C7210        5
C1257        5
C1600        4
C4100        4
C2400        4
C1800        3
C1260        3
C0           2
C1267        2
C1246        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [24]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value_class = 1000
classifications_to_replace = applications_class[applications_class < cutoff_value_class].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    12505
C2000     4681
C1200     3985
Other     1679
C2100     1617
C3000     1532
Name: CLASSIFICATION, dtype: int64

In [25]:
application_df = application_df.drop(['SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], axis = 1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,5000,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,6692,1
5,T3,Independent,C1200,Preservation,Trust,1,5000,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,5000,0
...,...,...,...,...,...,...,...,...
34293,T3,CompanySponsored,C1000,Preservation,Association,1,5000,1
34294,T4,Independent,C1000,ProductDev,Association,1,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,5000,0


In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
# application_df.dtypes
dummy_df = pd.get_dummies(application_df[['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION']])
merged_df = pd.concat([application_df, dummy_df], axis=1)
merged_df = merged_df.drop(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION'], axis=1)
merged_df.head()
merged_df.tail()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
34293,1,5000,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0


In [27]:
# Split our preprocessed data into our features and target arrays
y = merged_df['IS_SUCCESSFUL'].values
X = merged_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8)

<ipython-input-27-23d043116dff>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = merged_df.drop(['IS_SUCCESSFUL'],1).values


In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [29]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# number_input_features = len(X_train[0])
# hidden_nodes_layer1 =  3
# hidden_nodes_layer2 = 6

# nn = tf.keras.models.Sequential()

# # First hidden layer
# nn.add(
#     tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
# )

# # Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# # Check the structure of the model
# nn.summary()

In [30]:
# # Compile the model
# nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# # Train the model
# fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

In [33]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.6 MB/s eta 0:00:00


In [35]:
def create_model(hp):
    nn = tf.keras.models.Sequential()
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    # Allow kerastuner to decide number of neurons in first layer
    nn.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=26,
        step=2), activation=activation, input_dim=len(X_train[0])))
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=26,
            step=2),
            activation=activation))
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    return nn

import keras_tuner as kt
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    directory='./untitled_project',
    project_name='tuner_z.json')

In [36]:
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 41s]
val_accuracy: 0.751230776309967

Best val_accuracy So Far: 0.7540000081062317
Total elapsed time: 00h 24m 08s


In [38]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 11,
 'num_layers': 3,
 'units_0': 11,
 'units_1': 9,
 'units_2': 25,
 'units_3': 21,
 'units_4': 3,
 'units_5': 13,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [39]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

204/204 - 1s - loss: 0.5304 - accuracy: 0.7540 - 510ms/epoch - 2ms/step
Loss: 0.5304096937179565, Accuracy: 0.7540000081062317


In [ ]:
# # Evaluate the model using the test data
# model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [40]:
# Export our model to HDF5 file
best_model.save("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
